# 01_define_extremes.ipynb
### Standardize anomalies and define extremes

This notebook loads the data from [00_preprocessing.ipynb](00_preprocess.ipynb) (outlined in red below) and detects extremes values. Detrended anomalies are standardized by the monthly mean standard deviation and extreme values are identified using the 90th percentile at each spatial point.


![preprocess_flow](images/01_detect_extremes.png)


In [53]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### Import data
- preprocessed monthly averaged OISST v2.1 data created in [00_preprocessing.ipynb](00_preprocess.ipynb)


In [2]:
ds = xr.open_mfdataset('/burg/abernathey/users/hillary/oisst_v21/preprocessed_OISSTv21.nc')
ds

<xarray.Dataset>
Dimensions:                 (time: 478, lat: 720, lon: 1440)
Coordinates:
  * time                    (time) datetime64[ns] 1981-09-01 ... 2021-06-01
  * lat                     (lat) float32 -89.88 -89.62 -89.38 ... 89.62 89.88
  * lon                     (lon) float32 0.125 0.375 0.625 ... 359.6 359.9
Data variables: (12/13)
    ssta_notrend            (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    mean                    (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    trend                   (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    seas                    (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    Nino3                   (time) float64 dask.array<chunksize=(478,), meta=np.ndarray>
    Nino3_4                 (time) float64 dask.array<chunksize=(478,), meta=np.ndarray>
    ...                      ...
    ssta_Nino3              (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    ssta_Nino3_4            (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    ssta_Nino4              (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    ssta_notrend_noNino3    (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    ssta_notrend_noNino3_4  (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    ssta_notrend_noNino4    (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
Attributes:
    description:  preprocessed monthly averaged OISST v2.1 data

#### Standardize SST anomalies by dividing by the monthly standard deviatoin
This places equal variance on SSTa at all spatial points

In [56]:
# compute monthly standard deviation
stdev_notrend = ds.ssta_notrend.groupby(ds.time.dt.month).std()

# divide by standard deviation
ssta_stn_notrend = ds.ssta_notrend.groupby(ds.time.dt.month)/stdev_notrend


#### Calculate 90th percentile threshold from the origonal detrended SST anomalies

In [54]:
# Define extreme threshold as the 90th percentile
thresh = .9

# Rechunk time dim
if ssta_stn_notrend.chunks:
    ssta_stn_notrend = ssta_stn_notrend.chunk({'time': -1})
    
# Compute values of the monthly threshold
prct_notrend = ssta_stn_notrend.quantile(thresh, dim='time', keep_attrs=True, skipna=True)


#### Find where SSTA exceeds the 90th percentile threshold

In [57]:
# Take only the SSTa that exceeds the calculated monthly threshold
mhw_ssta_notrend = ds.ssta_notrend.where(ds.ssta_notrend>prct_notrend)
mhw_ssta_notrend_noNino3 = ds.ssta_notrend_noNino3.where(ds.ssta_notrend_noNino3 > prct_notrend)
mhw_ssta_notrend_noNino3_4 = ds.ssta_notrend_noNino3_4.where(ds.ssta_notrend_noNino3_4 > prct_notrend)
mhw_ssta_notrend_noNino4 = ds.ssta_notrend_noNino4.where(ds.ssta_notrend_noNino4 > prct_notrend)

### Save output 

In [61]:
# Specify path to save output 
path = '/burg/abernathey/users/hillary/oisst_v21/'

ds_new = xr.Dataset(
    data_vars=dict(
        ssta_stn_notrend=(['time', 'lat', 'lon'], ssta_stn_notrend.data),
        prct_notrend = (['lat', 'lon'], prct_notrend.data),
        
        mhw_ssta_notrend = (['time', 'lat', 'lon'], mhw_ssta_notrend.data),
        mhw_ssta_notrend_noNino3 = (['time', 'lat', 'lon'], mhw_ssta_notrend_noNino3.data),
        mhw_ssta_notrend_noNino3_4=(['time', 'lat', 'lon'], mhw_ssta_notrend_noNino3_4.data),
        mhw_ssta_notrend_noNino4=(['time', 'lat', 'lon'], mhw_ssta_notrend_noNino4.data),     
    ),
    coords=dict(
        time=(['time'], ds.time.data),
        lat=(['lat'], ds.lat.data),
        lon=(['lon'], ds.lon.data),
    ),
    attrs=dict(description="standardize SST anomalies and find positive extreme values", 
               threshold=thresh, 
               data='preprocessed monthly averaged OISST v2.1'),
)

ds_new.to_netcdf(path+'positive_extremes_OISSTv21.nc', mode='w')

In [62]:
ds_new

<xarray.Dataset>
Dimensions:                     (time: 478, lat: 720, lon: 1440)
Coordinates:
  * time                        (time) datetime64[ns] 1981-09-01 ... 2021-06-01
  * lat                         (lat) float32 -89.88 -89.62 ... 89.62 89.88
  * lon                         (lon) float32 0.125 0.375 0.625 ... 359.6 359.9
Data variables:
    ssta_stn_notrend            (time, lat, lon) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    prct_notrend                (lat, lon) float64 dask.array<chunksize=(720, 1440), meta=np.ndarray>
    mhw_ssta_notrend            (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    mhw_ssta_notrend_noNino3    (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    mhw_ssta_notrend_noNino3_4  (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
    mhw_ssta_notrend_noNino4    (time, lat, lon) float64 dask.array<chunksize=(478, 720, 1440), meta=np.ndarray>
Attributes:
    description:  standardize SST anomalies and find positive extreme values
    threshold:    0.9
    data:         preprocessed monthly averaged OISST v2.1